In [1]:
import os
import json
import mmcv
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob

In [2]:
# Calculate interval between tracklets of same video
# Open video and check number of frames - fill empty frames

In [3]:
def get_tracklet_df(annotation_path, video_path):
    # Generated by single_ape_subset dirs
    annotations = glob(f'{annotation_path}/**/*.json', recursive=True)
    video_ids = glob(f'{video_path}/**/*.mp4', recursive=True)
    
    ids, tracklet_num, idxs_st = [], [], []

    for a in annotations:
        with open(a, 'rb') as handle:
            data = json.load(handle)

        name = data['video']
        idxs = []

        for frame in data['annotations']:
            if(frame['detections'][0]['ape_id'] == 0):
                idxs.append(frame['frame_id'])

        tracklets = 1
        start_idxs = [idxs[0]]

        for i in range(len(idxs)-1):
            if(idxs[i+1] - idxs[i] == 1):
                continue
            else:
                tracklets+=1
                start_idxs.append(idxs[i])
                start_idxs.append(idxs[i+1])
        start_idxs.append(idxs[-1])

        ids.append(name)
        tracklet_num.append(tracklets)
        idxs_st.extend([start_idxs])
    
    df = pd.DataFrame([ids, tracklet_num, idxs_st]).transpose()
    df.columns = ['id', 'tracklet_num', 'indexes']
    return df

In [6]:
# Generated by single_ape_subset dirs
tracklet_df = get_tracklet_df(annotation_path='../single_ape_subset/annotations', video_path='../single_ape_subset/videos')

# Manual tracklet analysis
tracklet_analysis = pd.read_excel('tracklet_analysis_manual.xlsx')

In [7]:
tracklet_df

,id,tracklet_num,indexes
0,ACP0003bgj,1,"[1, 360]"
1,ACP000b6dx,1,"[1, 360]"
2,ACP000b3gy,1,"[1, 360]"
3,ACP000483i,1,"[1, 360]"
4,ACP000cc3p,1,"[1, 222]"
...,...,...,...
1961,ACP000ezdt,1,"[72, 348]"
1962,ACP00036on,1,"[1, 274]"
1963,ACP000b3ia,1,"[1, 133]"
1964,ACP0007gw6,1,"[1, 97]"


In [40]:
tracklet_analysis['interpolate'] = tracklet_analysis['interpolate'].apply(lambda x: True if x == 1 else False)
tracklet_analysis['first_tracklet'] = tracklet_analysis['first_tracklet'].apply(lambda x: True if x == 1 else False)
tracklet_analysis['second_tracklet'] = tracklet_analysis['second_tracklet'].apply(lambda x: True if x == 1 else False)

In [8]:
tracklet_analysis = tracklet_analysis.join(tracklet_df.set_index('id'), on='ID', how='outer')

In [9]:
tracklet_analysis

,Unnamed: 0,ID,notes,interpolate,first_tracklet,second_tracklet,in_out,tracklet_num,indexes
0.0,74.0,ACP00000ga,NaN,1.0,0.0,0.0,0.0,1,"[1, 257]"
1.0,132.0,ACP0000acj,Take first tracklet,0.0,1.0,0.0,0.0,1,"[2, 108]"
2.0,33.0,ACP00027g8,NaN,1.0,0.0,0.0,0.0,1,"[1, 360]"
3.0,34.0,ACP00029k4,NaN,1.0,0.0,0.0,0.0,1,"[13, 360]"
4.0,30.0,ACP00029mz,NaN,1.0,0.0,0.0,0.0,1,"[1, 359]"
...,...,...,...,...,...,...,...,...,...
NaN,NaN,ACP000bco6,NaN,NaN,NaN,NaN,NaN,1,"[1, 360]"
NaN,NaN,ACP000ezdt,NaN,NaN,NaN,NaN,NaN,1,"[72, 348]"
NaN,NaN,ACP000b3ia,NaN,NaN,NaN,NaN,NaN,1,"[1, 133]"
NaN,NaN,ACP0007gw6,NaN,NaN,NaN,NaN,NaN,1,"[1, 97]"


In [10]:
tracklet_analysis[tracklet_analysis.tracklet_num!=1]['ID']

17.0     ACP0003dym
18.0     ACP0003t3z
26.0     ACP00055oz
32.0     ACP0005l7v
38.0     ACP0005w4w
117.0    ACP000bu9t
126.0    ACP000cefw
135.0    ACP000f2tg
141.0    ACP000gbhs
Name: ID, dtype: object

In [12]:
annotation_path='../single_ape_subset/annotations'
video_path='../single_ape_subset/videos'
for value in tracklet_analysis[tracklet_analysis.tracklet_num!=1]['ID'].values:
    os.remove(f"{annotation_path}/{value}.json")
    os.remove(f"{video_path}/{value}.mp4")

In [ ]:
tracklet_analysis[(tracklet_analysis['interpolate']==False) & (tracklet_analysis['first_tracklet']==False) | (tracklet_analysis['second_tracklet']==False)]

In [ ]:
outpath = '/home/dl18206/Desktop/phd/data/panaf/single_ape_subset/annotations'
annotation_prefix = '/home/dl18206/Desktop/phd/data/panaf/single_ape_subset/annotations'
# Index(['interpolate', 'first_tracklet', 'second_tracklet'], dtype='object')

for column in tracklet_analysis.columns[3:6]:
    ids = tracklet_analysis[tracklet_analysis[column]].ID.values
    
    for elem in tqdm(ids):
        with open(f"{annotation_prefix}/{elem}.json", 'rb') as handle:
            data = json.load(handle)
        
        idxs = tracklet_analysis[tracklet_analysis.ID == elem].indexes.values[0]
        
        if column == 'interpolate':
            for i in range(1, len(idxs)-1, 2):
                frame1 = get_frame(data, idxs[i])
                frame2 = get_frame(data, idxs[i+1])
                data = interpolate_between_frames(frame1, frame2)
        
        elif column == 'first_tracklet':
            start, end = get_index_pairs(idxs, 'first')
            abs_s, abs_e = convert2idxs(start, end)
            data['annotations'] = data['annotations'][abs_s: abs_e]
            
        elif column == 'second_tracklet':
            start, end = get_index_pairs(idxs, 'second')
            abs_s, abs_e = convert2idxs(start, end)
            data['annotations'] = data['annotations'][abs_s: abs_e]            

        with open(f'{outpath}/{elem}.json', 'w') as handle:
            json.dump(data, handle, ensure_ascii=False, indent=4)

In [ ]:
tracklet_analysis.columns[3:6]

In [ ]:
# Generated by single_ape_subset dirs
tracklet_df = get_tracklet_df(annotation_path, video_path)

In [ ]:
tracklet_df[tracklet_df.tracklet_num!=1]

**Interpolation: single video test**

In [ ]:
def get_frame(data, frame_idx):
    f = None
    for frame in data['annotations']:
        if(frame['frame_id'] == frame_idx):
            f = frame
    return f

In [ ]:
def interpolate_bbox(bbox1, bbox2, gap):
    xmin  = np.linspace(bbox1[0], bbox2[0], gap)
    ymin  = np.linspace(bbox1[1], bbox2[1], gap)
    xmax  = np.linspace(bbox1[2], bbox2[2], gap)
    ymax  = np.linspace(bbox1[3], bbox2[3], gap)
    return xmin, ymin, xmax, ymax

In [ ]:
def interpolate_between_frames(frame1, frame2):
    gap = frame2['frame_id'] - frame1['frame_id'] - 1

    bbox1 = frame1['detections'][0]['bbox']
    bbox2 = frame2['detections'][0]['bbox']

    site = frame2['detections'][0]['site']
    species = frame2['detections'][0]['species']

    scores = np.linspace(
        frame1['detections'][0]['detection_score'], 
        frame2['detections'][0]['detection_score'],
        gap
    )

    xmin, ymin, xmax, ymax = interpolate_bbox(bbox1, bbox2, gap)

    for i, (x1, y1, x2, y2, score) in enumerate(zip(xmin, ymin, xmax, ymax, scores), start=frame1['frame_id']+1):
        f = dict(
            frame_id=i,
            detections=[dict(
                ape_id=0, 
                site=site,
                species=species,
                detection_score=score,
                bbox=[x1, y1, x2, y2]
            )]
        )
        data['annotations'].append(f)
        
    data['annotations'] = sorted(data['annotations'], key=lambda x:x['frame_id'], reverse=False)
    return data

In [ ]:
test_ann = '/home/dl18206/Desktop/phd/data/panaf/single_ape_subset/annotations/ACP000cbk4.json'
test_idxs = [1, 108, 160, 360]

with open(test_ann, 'rb') as handle:
    data = json.load(handle)

for i in range(1, len(test_idxs)-1, 2):
    frame1 = get_frame(data, test_idxs[i])
    frame2 = get_frame(data, test_idxs[i+1])
    data = interpolate_between_frames(frame1, frame2)

In [ ]:
# get_frame(data, 250)

In [ ]:
# with open('ACP000cbk4.json', 'w') as handle:
#     json.dump(data, handle, ensure_ascii=False, indent=4)

**Remove tracklet**

In [ ]:
test_ann = '/home/dl18206/Desktop/phd/data/panaf/single_ape_subset/annotations/ACP000cbk4.json'
test_idxs = [1, 108, 160, 360]

In [ ]:
def get_index_pairs(test_idxs, which):
    idxs = None
    if(which=='first'):
        idxs = test_idxs[0], test_idxs[1]
    elif(which=='second'):
        idxs = test_idxs[2], test_idxs[3]
    return idxs

In [ ]:
with open(test_ann, 'rb') as handle:
    data = json.load(handle)

In [ ]:
data['annotations'][308]

In [ ]:
def convert2idxs(start_idx, end_idx):
    abs_idxs = []
    for i, frame in enumerate(data['annotations']):
        if(frame['frame_id'] == start_idx):
            abs_idxs.append(i)
        elif(frame['frame_id'] == end_idx):
            abs_idxs.append(i+1)
    return abs_idxs

In [ ]:
start, end = get_index_pairs(test_idxs, 'second')
abs_s, abs_e = convert2idxs(start, end)
abs_s, abs_e

In [ ]:
data['annotations'][108:309]